In [1]:
import hs_alkane.alkane as mdl
import numpy as np
import sys
import os
import copy


# For progress bar
from ipywidgets import IntProgress
from IPython.display import display

#visualise data
import matplotlib.pyplot as plt

aspect_ratio_limit = 0.8
angle_limit_deg = 45

angle_limit_rad = angle_limit_deg*2*np.pi/360


nwalkers = 500
active_box = nwalkers+1
nchains = 32
nbeads = 1
max_vol_per_atom = 15
mdl.box_set_num_boxes(nwalkers+1)
mdl.box_initialise()
mdl.box_set_pbc(1)
mdl.alkane_set_nchains(nchains) 
mdl.alkane_set_nbeads(nbeads)    
mdl.alkane_initialise()           
mdl.box_set_isotropic(1)
mdl.box_set_bypass_link_cells(1) # Bypass use of link cell algorithm for neighbour finding
mdl.box_set_use_verlet_list(0)   # Don't use Verlet lists either since CBMC moves quickly invalidate these

#random seeds
np.random.seed(10)
mdl.random_set_random_seed(10)

cell_matrix = 0.999*np.eye(3)*np.cbrt(nbeads*nchains*max_vol_per_atom)#*np.random.uniform(0,1)
for ibox in range(1,nwalkers+1):
    mdl.box_set_cell(ibox,cell_matrix)


    

In [2]:

def box_shear_step(ibox, step_size):
    # pick random vector for shear direction
    #np.random.seed(10)
    rnd_vec_ind = np.random.randint(0, 3)
    # turn other two into orthonormal pair
    #should I have a pair or should I have a single vector.
    other_vec_ind = list(range(3))
    other_vec_ind.remove(rnd_vec_ind)
    orig_cell = copy.deepcopy(mdl.box_get_cell(ibox))
    orig_cell_copy = copy.deepcopy(orig_cell)

    
    v1 = orig_cell_copy[other_vec_ind[0],:]
    v2 = orig_cell_copy[other_vec_ind[1],:]
    
    v1 /= np.sqrt(np.dot(v1,v1))
    v2 -= v1*np.dot(v1,v2) 
    v2 /= np.sqrt(np.dot(v2,v2))

    
    # pick random magnitudes
    rv1 = np.random.uniform(high = step_size)
    rv2 = np.random.uniform(high = step_size)
#     rv1 = 2
#     rv2 = 0

    # create new cell and transformation matrix (matrix is additive)
    new_cell = orig_cell.copy()

    new_cell[rnd_vec_ind,:] += rv1*v1 + rv2*v2

    delta_H = new_cell - orig_cell
    
    #reject due to poor aspect ratio

    

    
    mdl.alkane_change_box(5,delta_H)
    #transform = np.dot(np.linalg.inv(orig_cell), new_cell)
    if min_aspect_ratio(new_cell) < aspect_ratio_limit:
        boltz = 0
    elif min_angle(new_cell) < angle_limit_rad:
        boltz = 0
    elif mdl.alkane_check_chain_overlap(ibox):
        boltz = 0   
    else:
        boltz = 1
    
    #bake rejection  due to shape in here as it becomes easier to fit with the rest of the code
    
    
    return boltz, delta_H

In [3]:
def box_stretch_step(ibox, step_size):
    cell = mdl.box_get_cell(ibox)
    new_cell = cell.copy()
    rnd_v1_ind = np.random.randint(0, 3)
    rnd_v2_ind = np.random.randint(0, 3)
    if rnd_v1_ind == rnd_v2_ind:
        rnd_v2_ind = (rnd_v2_ind+1) % 3

    rv = 1+np.random.uniform(0.0,0.5)
    print(rv)
    #rv = 1+0.5
    #transform = np.eye(3)
    new_cell[rnd_v1_ind,rnd_v1_ind] *= rv
    new_cell[rnd_v2_ind,rnd_v2_ind] *= (1/rv)
    
    delta_H = new_cell - cell
    print(delta_H)
    

    
    mdl.alkane_change_box(5,delta_H)
    #transform = np.dot(np.linalg.inv(orig_cell), new_cell)
    if min_aspect_ratio(new_cell) < aspect_ratio_limit:
        boltz = 0
    elif min_angle(new_cell) < angle_limit_rad:
        boltz = 0
    elif mdl.alkane_check_chain_overlap(ibox):
        boltz = 0   
    else:
        boltz = 1
    
    
    return (boltz, delta_H)

In [ ]:
print(box_stretch_step(5,1))

In [4]:
vis_chains(mk_ase_config(5,nchains,nbeads))

NameError: name 'vis_chains' is not defined

In [5]:
def min_aspect_ratio(cell):
    vol = np.prod(cell.diagonal())
    #returns the shortest distance between two parallel faces, scaled such that the cell has a volume of 1
    min_aspect_ratio = sys.float_info.max
    for i in range(3):
        vi = cell[i,:]
        vnorm_hat = np.cross(cell[(i+1)%3,:],cell[(i+2)%3,:])
        vnorm_hat = vnorm_hat/(np.sqrt(np.dot(vnorm_hat,vnorm_hat)))
        min_aspect_ratio = min(min_aspect_ratio, abs(np.dot(vnorm_hat,vi)))
    return min_aspect_ratio/(vol**(1.0/3.0))


In [6]:
def min_angle(cell):
    min_angle = sys.float_info.max
    for i in range(3):
        vc1 = cell[(i+1)%3,:]
        vc2 = cell[(i+2)%3,:]
        dot_prod = np.dot(vc1,vc2)/np.sqrt((np.dot(vc1,vc1))*(np.dot(vc2,vc2)))
        if dot_prod < 0:
            dot_prod *= -1
        vec_angle = np.arccos(dot_prod)
        min_angle = min(min_angle,vec_angle)
    return vec_angle
        

In [7]:
from ase import Atoms
from ase.visualize import view
import ase

def mk_ase_config(ibox, Nbeads, Nchains):
    'Uses the current state of the alkane model to construct an ASE atoms object'
    
    # Create and populate ASE object
    model_positions = np.empty([Nchains*Nbeads, 3])
    cell_vectors = mdl.box_get_cell(ibox)
   
    for ichain in range(0, Nchains):
        model_positions[Nbeads*ichain:Nbeads*ichain+Nbeads] = mdl.alkane_get_chain(ichain+1, ibox)
    
    confstring = "C"+str(Nbeads*Nchains)
    
    box_config = Atoms(confstring, positions=model_positions*(1.5/0.4), pbc=True, cell=cell_vectors*(1.5/0.4))


    return box_config  # Returns ASE atom object

In [8]:
ncopy = nchains
for ibox in range(1,nwalkers+1):
    for ichain in range(1,ncopy+1):
        rb_factor = 0
        mdl.alkane_set_nchains(ichain)
        overlap_flag = 1
        while overlap_flag != 0 and ichain != 1:
            rb_factor, ifail = mdl.alkane_grow_chain(ichain,ibox,1)         
#             if ifail != 0:
#                 rb_factor = 0
            overlap_flag = mdl.alkane_check_chain_overlap(ibox)

        


In [9]:
atoms = mk_ase_config(13, nbeads, nchains)
#atoms = ase.io.read("test.traj")
view(atoms, viewer='nglview')


In [ ]:
mdl.alkane_set_dr_max(0.65)
mdl.alkane_set_dt_max(0.43)
mdl.alkane_set_dh_max(0.4)
mdl.alkane_set_dv_max(0.5)

In [ ]:
def vis_chains(vis_config):
    '''Takes an ASE atoms object or list thereof and creates a customised ngl viewer 
       with appropriate settings for our bulk alkane chain models'''

    met = 0.35
    rad = 1.0
    
    colours = ['#DDDDDD']#['#FF1111','#FFAAAA', '#DDDDDD', '#1111FF', '#AAAAFF']
    ncols = len(colours)
    
    try:
            
        vis_config.wrap()
    except:
        pass
    
    sel=list()
    for icol in range(ncols):
        sel.append(list())
    
    # Create lists for each colour
    for ichain in range(0, nchains):
        icol = ichain%ncols
        for ibead in range(nbeads):
            iatom = ichain*nbeads + ibead
            sel[icol].append(iatom)
            
    v = view(vis_config, viewer='ngl')                   
    v.view.clear_representations()
    v.view.add_representation('unitcell', color='#000000')
    
    for icol in range(ncols):
        v.view.add_representation('ball+stick', selection=sel[icol], color=colours[icol], radius=rad, metalness=met)

    return v

In [ ]:
def MC_run(sweeps, move_ratio, ibox, volume_limit = sys.float_info.max):
    moves_accepted = np.zeros(4)
    moves_attempted = np.zeros(4)
    nbeads = mdl.alkane_get_nbeads()
    nchains = mdl.alkane_get_nchains()
    isweeps = 0
    pressure = 0
    move_prob = np.cumsum(move_ratio)/np.sum(move_ratio)
    while isweeps < sweeps:
        imove=0
        while imove< nchains+1:
            ichain = np.random.randint(0, high=nchains) # picks a chain at random
            #should it be from 0 to nchains?
            current_chain = mdl.alkane_get_chain(ichain+1, ibox)
            backup_chain = current_chain.copy()
            xi = np.random.random()
            if xi < move_prob[ivol]:
                # Attempt a volume move
                itype = ivol
                boltz = mdl.alkane_box_resize(pressure, ibox, 0)
                moves_attempted[itype] += 1
            elif xi < move_prob[itrans]:
                # Attempt a translation move
                itype = itrans
                boltz = mdl.alkane_translate_chain(ichain+1, ibox)
                moves_attempted[itrans] += 1
            elif xi < move_prob[irot]:
                # Attempt a rotation move
                itype = irot
                boltz, quat = mdl.alkane_rotate_chain(ichain+1, ibox, 0)
                moves_attempted[itype] += 1
            else:
                # Attempt a dihedral angle move
                itype = idih
                boltz, bead1, angle = mdl.alkane_bond_rotate(ichain+1, ibox, 1)
                moves_attempted[itype] += 1

            if (np.random.random() < boltz):
                #accept the move
                moves_accepted[itype]+=1

#             else:
                
#                 # Reject move
#                 if (itype!=ivol):
#                     # Restore old chain if single chain move
#                     for ibead in range(nbeads):
#                         current_chain[ibead] = backup_chain[ibead]
#                 else:
#                     # Reset the box change - special fucntion for this.
#                     dumboltz = mdl.alkane_box_resize(pressure, ibox, 1)
                    
                    
                    
            if (itype==ivol):
                new_volume = mdl.box_compute_volume(ibox)
                if (np.random.random() < boltz) and (new_volume - volume_limit) < sys.float_info.epsilon:
                    moves_accepted[itype]+=1
                else:
                    dumboltz = mdl.alkane_box_resize(pressure, ibox, 1)

                
            else:
                if (np.random.random() < boltz):
                #accept the move
                    moves_accepted[itype]+=1
                else:
                    #reject the move
                    for ibead in range(nbeads):
                        current_chain[ibead] = backup_chain[ibead]


            imove += 1
        isweeps +=1
    moves_attempted = np.where(moves_attempted == 0, 1, moves_attempted)
    moves_acceptance_rate = moves_accepted/moves_attempted

    return mdl.box_compute_volume(ibox), moves_acceptance_rate

In [ ]:
move_types = ['box','translate', 'rotate', 'dihedral']
ivol = 0; itrans = 1; irot = 2; idih = 3
move_ratio = np.zeros(4)
move_ratio[ivol] = 6
move_ratio[itrans] = 3.0*nchains
#moves_ratio[irot] = 2.0*nchains
#moves_ratio[idih] = 1.0*nchains
#moves_prob = np.cumsum(moves_ratio)/np.sum(moves_ratio)
walk_length = 20


In [ ]:
volumes = {}
start_volumes = []
for ibox in range(1,nwalkers+1):
    volumes[ibox], rate = MC_run(walk_length, move_ratio, ibox)
    start_volumes.append(volumes[ibox])


In [ ]:
overlap_check = np.zeros(nwalkers)
for ibox in range(1,nwalkers+1):
    overlap_check[mdl.alkane_check_chain_overlap(ibox)]
print(np.where(overlap_check!=0))

In [ ]:
atoms = mk_ase_config(5, nbeads, nchains)
vis_chains(atoms)

In [ ]:
def clone_walker(ibox_original,ibox_clone):
    cell = mdl.box_get_cell(ibox_original)
    mdl.box_set_cell(ibox_clone,cell)
    for ichain in range(1,nchains+1):
        original_chain = mdl.alkane_get_chain(ichain,ibox_original)
        original_chain_copy = original_chain.copy()
        clone_chain = mdl.alkane_get_chain(ichain,ibox_clone)
        for i in range(nbeads):
            clone_chain[i][:] = original_chain_copy[i][:]
            

In [ ]:
def adjust_dv(ibox,active_box, lower_bound,upper_bound):
    equil_factor = 2
    sweeps = 20
    move_ratio = [1,0,0,0]
    clone_walker(ibox,active_box)
    vol,acceptance_rate = MC_run(sweeps, move_ratio, active_box)
    
    r = acceptance_rate[ivol]
    
    if r < lower_bound:
        mdl.alkane_set_dv_max(mdl.alkane_get_dr_max()/equil_factor)
    elif r > upper_bound:
        mdl.alkane_set_dv_max(mdl.alkane_get_dr_max()*equil_factor)
    return r

    

In [ ]:
def adjust_dr(ibox,active_box, lower_bound,upper_bound):
    equil_factor = 2
    sweeps = 20
    move_ratio = [0,1,0,0]
    clone_walker(ibox,active_box)
    vol,acceptance_rate = MC_run(sweeps, move_ratio, active_box)
    
    r = acceptance_rate[itrans]
    
    if r < lower_bound:
        mdl.alkane_set_dr_max(mdl.alkane_get_dr_max()/equil_factor)
    elif r > upper_bound:
        mdl.alkane_set_dr_max(mdl.alkane_get_dr_max()*equil_factor)
    return r

    

In [ ]:
def adjust_dt(ibox,active_box, lower_bound,upper_bound):
    equil_factor = 2
    sweeps = 20
    move_ratio = [0,0,1,0]
    clone_walker(ibox,active_box)
    vol,acceptance_rate = MC_run(sweeps, move_ratio, active_box)
    
    r = acceptance_rate[irot]
    
    if r < lower_bound:
        mdl.alkane_set_dt_max(mdl.alkane_get_dr_max()/equil_factor)
    elif r > upper_bound:
        mdl.alkane_set_dt_max(mdl.alkane_get_dr_max()*equil_factor)
    return r

    

In [ ]:
def adjust_dh(ibox,active_box, lower_bound,upper_bound):
    equil_factor = 2
    sweeps = 20
    move_ratio = [0,0,0,1]
    clone_walker(ibox,active_box)
    vol,acceptance_rate = MC_run(sweeps, move_ratio, active_box)
    
    r = acceptance_rate[idih]
    
    if r < lower_bound:
        mdl.alkane_set_dr_max(mdl.alkane_get_dh_max()/equil_factor)
    elif r > upper_bound:
        mdl.alkane_set_dr_max(mdl.alkane_get_dh_max()*equil_factor)
    return r
 
    

In [ ]:
ns_iterations = 100000
walk_length = 100

removed_volumes = []


mc_adjust_interval = nwalkers//2 #for adjusting step sizes
low_acc_rate = 0.2
high_acc_rate = 0.5

snapshots = 1000

vis_interval = ns_iterations//snapshots


try:
    os.remove("nestedsampling.traj")
except:
    pass


traj = ase.io.Trajectory("nestedsampling.traj", mode="a")


f = IntProgress(min=0, max=ns_iterations) 
display(f) # display the bar

for i in range(ns_iterations):
    index_max = max(volumes, key=volumes.get)
    volume_limit = volumes[index_max]
    clone = np.random.randint(1,nwalkers)
    
    if i%vis_interval == 0:
        largest_config = mk_ase_config(index_max,nbeads,nchains)
        largest_config.wrap()
        traj.write(largest_config)

    
#     if i%mc_adjust_interval == 0:
#         adjust_dv(clone,active_box,low_acc_rate,high_acc_rate)
#         r = adjust_dr(clone,active_box,low_acc_rate,high_acc_rate)
# #         adjust_dt(clone,active_box,low_acc_rate,high_acc_rate)
# #         adjust_dh(clone,active_box,low_acc_rate,high_acc_rate)
    
    clone_walker(clone,active_box) #copies the ibox from first argument onto the second one.
    
    
    new_volume,_ = MC_run(walk_length, move_ratio, active_box,volume_limit)
    
    
    if new_volume-volume_limit<sys.float_info.epsilon:
        removed_volumes.append(volumes[index_max])
        volumes[index_max] = new_volume #replacing the highest volume walker
        clone_walker(active_box, index_max)
    print(volume_limit)

        
    
    

    f.value+=1

traj.close()

Expected packing for 32 spheres is $\approx$ 22.3

Two approaches to cloning a simulation cell for generating new configurations.

1. Write a random configuration to an xyz file, and then load it into an extra empty walker, which will be used to perform trial moves, and then save as an xyz file. Then you can use the highest volume actual walker to load the positions, and continue. Issue is, loading and saving will be the brunt of the calculation.

2. Use a numpy array to hold positions (i.e use it as a backup), and then manually change one of the walkers to be the clone, by setting positions and unit cell. Can get very messy, but should be much faster.

3. Blend the methods. Use an extra walker to hold the atoms but enter the positions and cell vectors from numpy array

In [ ]:
atoms = mk_ase_config(6, nbeads, nchains)
v = vis_chains(atoms)
# atoms.wrap()
# v = view(atoms,viewer='nglview')

v

In [ ]:
vis_traj = ase.io.read("nestedsampling.traj", index = ':')

#v = view(vis_traj, viewer = 'mpl')



vis_chains(vis_traj)

In [ ]:
overlap_check = np.zeros(nwalkers)
for ibox in range(1,nwalkers+1):
    overlap_check[mdl.alkane_check_chain_overlap(ibox)]
print(np.where(overlap_check!=0))

In [ ]:
#distribution of volumes shown as a histogram
end_volumes = []
for ibox in range(1,nwalkers+1):
    end_volumes.append(volumes[ibox])
#plt.hist(start_volumes, bins = 20)
#plt.hist(end_volumes)
plt.show()

In [ ]:
#plot of removed volumes in the order they are removed, excluding final volumes
plt.plot(removed_volumes)
#plt.yscale("log")
plt.title("Volume as a function of iterations ran")
plt.show()

In [ ]:
#
#rdf functionality
def py_rdf(r,S, dr, dim):
    from numpy import zeros, sqrt, where, pi, mean, arange, histogram, absolute
    num_particles  = len(r)
    rMax           = S/2.0
    edges          = arange(0, rMax + dr, dr)
    num_increments = len(edges) - 1
    g              = zeros(num_increments)
    radii          = zeros(num_increments)
    numberDensity  = len(r) / atoms.get_volume()

    # Compute pairwise correlation for each particle
    for index in range(num_particles):

        d = 0.0
        for i in range(dim):
            dp = absolute(r[index,i] - r[:,i])
            mask = dp>S/2.0
            dp[mask] = S - dp[mask]
            d += dp*dp

        d = sqrt(d)
        d[index] = 2 * rMax

        (result, bins) = histogram(d, bins=edges, density=False)
        g += result

    g = g/(num_particles * numberDensity)

    # Normalize the g(r) dividing by the g(r) of an ideal gas - in 2D!
    if dim == 2:
        for i in range(num_increments):
            radii[i] = (edges[i] + edges[i+1]) / 2.
            rOuter = edges[i + 1]
            rInner = edges[i]
            g[i] = g[i] / (2.0 * pi * (rOuter-rInner)* radii[i])
    
    # Needed to compute the 3D g(r) (blue box)
    # Normalize the g(r) divinding by the g(r) of an ideal gas - in 3D!
    if dim == 3:
        for i in range(num_increments):
            radii[i] = (edges[i] + edges[i+1]) / 2.
            rOuter = edges[i + 1]
            rInner = edges[i]
            g[i] = g[i] / (4.0 * pi * (rOuter-rInner)* radii[i] * radii[i] )

    return (radii, g)

In [ ]:
mdl.alkane_change_box?

In [ ]:
r = atoms.positions
S = 16
dim = 3
dr = 0.2
S = np.amax(atoms.cell)



radii, g = py_rdf(r,S, dr, dim)

In [ ]:
plt.plot(radii,g)
plt.show()

In [ ]:
#shear and stretch moves

def shear_move(ibox):
    pass
def stretch_move(ibox):
    pass
    

In [ ]:
t_cell = np.array([[10,0,0],[0,10,0],[0,0,10]])

vol = np.prod(t_cell.diagonal())
min_theta = min_angle(t_cell)
print(min_theta) 

print(360*min_theta/(2*np.pi))

In [ ]:
mdl.box_set_cell(6,t_cell)

In [ ]:
atoms = mk_ase_config(6, nbeads, nchains)
v = vis_chains(atoms)
# atoms.wrap()
# v = view(atoms,viewer='nglview')

v

In [ ]:
np.sqrt(3/4)